# Safe to eat or deadly poisonous?
### An analysis on mushroom classification by Lorenzo Santolini

#### Code snippet for google colab
This is a little code to import automatically the dataset into google colab. Provide your kaggle's API key (profile section) when file requested

In [14]:

# Little code snippet to import on Google Colab the dataset
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

# Insert here your kaggle API key
from google.colab import files
files.upload()

!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d uciml/mushroom-classification
!unzip mushroom-classification.zip
!ls


Saving kaggle.json to kaggle (1).json
mushroom-classification.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  mushroom-classification.zip
replace mushrooms.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
'kaggle (1).json'   mushroom-classification.zip   sample_data
 kaggle.json	    mushrooms.csv


## Introduction
This dataset includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family Mushroom. Each species is identified as edible or poisonous. Rows are composed by 23 different fields, each one of them identifying a specific charateristic:

- Class: poisonous=p, edible=e
- Cap-shape: bell=b, conical=c, convex=x, flat=f, knobbed=k, sunken=s
- Cap-surface: fibrous=f, grooves=g, scaly=y, smooth=s
- Cap-color: brown=n, buff=b, cinnamon=c, gray=g, green=r, pink=p, purple=u, red=e, white=w, yellow=y
- Bruises: bruises=t, no=f
- Odor: almond=a, anise=l, creosote=c, fishy=y, foul=f, musty=m, none=n, pungent=p, spicy=s
- Gill-attachment: attached=a, descending=d, free=f, notched=n
- Gill-spacing: close=c, crowded=w, distant=d
- Gill-size: broad=b, narrow=n
- Gill-color: black=k, brown=n, buff=b, chocolate=h, gray=g, green=r, orange=o, pink=p, purple=u,red=e, white=w, yellow=y
- Stalk-shape: enlarging=e, tapering=t
- Stalk-root: bulbous=b, club=c, cup=u, equal=e, rhizomorphs=z, rooted=r, missing=?
- Stalk-surface-above-ring: fibrous=f, scaly=y, silky=k, smooth=s
- Stalk-surface-below-ring: fibrous=f, scaly=y, silky=k, smooth=s
- Stalk-color-above-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
- Stalk-color-below-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
- Veil-type: partial=p, universal=u
- Veil-color: brown=n, orange=o, white=w, yellow=y
- Ring-number: none=n, one=o, two=t
- Ring-type: cobwebby=c, evanescent=e, flaring=f, large=l, none=n, pendant=p, sheathing=s, zone=z
- Spore-print-color: black=k, brown=n, buff=b, chocolate=h, green=r, orange=o, purple=u, white=w, yellow=y
- Population: abundant=a, clustered=c, numerous=n, scattered=s, several=v, solitary=y
- Habitat: grasses=g, leaves=l, meadows=m, paths=p, urban=u, waste=w, woods=d

This analysis was conducted in Python 3.7.1 using Jupyter Notebook allows you to combine code, comments, multimedia, and visualizations in an interactive document — called a notebook, naturally — that can be shared, re-used, and re-worked. In addition, the following packages were used:

- sklearn
- pandas
- numpy
- plotly


In [0]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import LabelEncoder

import plotly
from plotly.plotly import plot, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff

plotly.tools.set_credentials_file(username='XXXXXXXX', api_key='XXXXXXXXXXXXXX')

## Dataset analysis and preprocessing
Let's start importing the data:

In [3]:
# Load the dataset
# dataset = pd.read_csv("./Input/mushrooms.csv")
dataset = pd.read_csv("./mushrooms.csv")
print("The dataset has %d rows and %d columns." % dataset.shape)

The dataset has 8124 rows and 23 columns.


Now we will look at the dataset to understand what are the different fields and their types:

In [4]:
# Count number of classes for classification
print(f"There are {dataset['class'].unique().size} different classes:"
      f"\n {dataset['class'].unique().tolist()}")

# Count number of unique data for every column
print(f"Unique values for every field: \n{dataset.nunique()}")


There are 2 different classes:
 ['p', 'e']
Unique values for every field: 
class                        2
cap-shape                    6
cap-surface                  4
cap-color                   10
bruises                      2
odor                         9
gill-attachment              2
gill-spacing                 2
gill-size                    2
gill-color                  12
stalk-shape                  2
stalk-root                   5
stalk-surface-above-ring     4
stalk-surface-below-ring     4
stalk-color-above-ring       9
stalk-color-below-ring       9
veil-type                    1
veil-color                   4
ring-number                  3
ring-type                    5
spore-print-color            9
population                   6
habitat                      7
dtype: int64


In [5]:
# See data types 
print(f"Data types: \n{dataset.head(5)}")

# All columns 
print(", ".join(str(a) for a in dataset.columns))

Data types: 
  class cap-shape cap-surface cap-color bruises odor gill-attachment  \
0     p         x           s         n       t    p               f   
1     e         x           s         y       t    a               f   
2     e         b           s         w       t    l               f   
3     p         x           y         w       t    p               f   
4     e         x           s         g       f    n               f   

  gill-spacing gill-size gill-color   ...   stalk-surface-below-ring  \
0            c         n          k   ...                          s   
1            c         b          k   ...                          s   
2            c         b          n   ...                          s   
3            c         n          n   ...                          s   
4            w         b          k   ...                          s   

  stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
0                      w                      w  

From the above snippet we can notice that the fields are all string values; converting them to numeric values can make our analysis much easier. We will take care of this in the next phase.

## Preprocessing
We need to pre-process our data to 
#### Handling missing values

In [18]:
# Check if any field is null
if dataset.isnull().any().any():
    print("There are some null values")
else:
    print("There are no null values")

There are no null values


It may seem that we have no missing value from the previous analysis, but if we look better,from the data description we can notice that in the field stalk-root there are some missing values, marked with the question mark; let's count how many of them there are:


In [19]:
print("There are " + str((dataset['stalk-root'] == "?").sum()) + " missing values in stalk-root column")

There are 2480 missing values in stalk-root column


When we find missing values in a dataset, there are some of the approaches that can be considered:

1. Delete all rows containing a missing value
2. Substitute with a constant value that has meaning within the domain, such as 0, distinct from all other values.
3. Substitute with a value from another randomly selected record.
4. Substitute with mean, median or mode value for the column.
5. Substitute with a value estimated by another predictive model.

It is evident from the `dataset.head()` function that our fileds are composed by all string values. Given the fact that we would need to translate in any case every field to a numeric one, to better display them in graphs, a simple approach is to keep the missing data as a peculiar number different from the others, and simply apply the transformation as they were present.

We will use the LabelEncoder from the sklearn library, which allows us to perform this mapping:

In [0]:
def preprocess(dataset):
  pre_data = dataset.copy()
  labelEncoder = LabelEncoder()
  for column in dataset.columns:
    dataset[column] = labelEncoder.fit_transform(dataset[column])
  return pre_data, le

In [21]:
le = 0
pre_data, le = preprocess(dataset)
# Check new data
pre_data.head(5)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [22]:

# Check new labels
print(pre_data.groupby('class').size())

class
e    4208
p    3916
dtype: int64


We can notice that data have been transformed, and now the labels are represented with a 0/1 integer value. 
Now we can look deeper into some statistical details about the dataset, using the `dataset.describe` command on our pandas DataFrame dataset. The output describes:

- count: number of samples (rows)
- mean: the mean of the attribute among all samples
- std: the standard deviation of the attribute
- min: the minimal value of the attribute
- 25%: the lower percentile
- 50%: the median
- 75%: the upper percentile
- max: the maximal value of the attribute

In [23]:
# Get insights on the dataset
dataset.describe()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,...,8124.000000,8124.000000,8124.000000,8124.0,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000
mean,0.482029,3.348104,1.827671,4.504677,0.415559,4.144756,0.974151,0.161497,0.309207,4.810684,...,1.603644,5.816347,5.794682,0.0,1.965534,1.069424,2.291974,3.596750,3.644018,1.508616
std,0.499708,1.604329,1.229873,2.545821,0.492848,2.103729,0.158695,0.368011,0.462195,3.540359,...,0.675974,1.901747,1.907291,0.0,0.242669,0.271064,1.801672,2.382663,1.252082,1.719975
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,3.000000,0.000000,2.000000,1.000000,0.000000,0.000000,2.000000,...,1.000000,6.000000,6.000000,0.0,2.000000,1.000000,0.000000,2.000000,3.000000,0.000000
50%,0.000000,3.000000,2.000000,4.000000,0.000000,5.000000,1.000000,0.000000,0.000000,5.000000,...,2.000000,7.000000,7.000000,0.0,2.000000,1.000000,2.000000,3.000000,4.000000,1.000000
75%,1.000000,5.000000,3.000000,8.000000,1.000000,5.000000,1.000000,0.000000,1.000000,7.000000,...,2.000000,7.000000,7.000000,0.0,2.000000,1.000000,4.000000,7.000000,4.000000,2.000000
max,1.000000,5.000000,3.000000,9.000000,1.000000,8.000000,1.000000,1.000000,1.000000,11.000000,...,3.000000,8.000000,8.000000,0.0,3.000000,2.000000,4.000000,8.000000,5.000000,6.000000


In [7]:
y = dataset["class"].value_counts()
print(y)

e    4208
p    3916
Name: class, dtype: int64
